### **README**

authorship: NestleJsCrunch, eric-lyons

# What this does

This creates a list of all dashboards currently on the Looker instance based on their `preferred_viewer` parameter. `preferred_viewer` controls which type the dashboard is, whether that be the new dashboard experience (`dashboards-next`) or legacy. 

# Notes
- Be warned: This may take a _very_ long time to run. Depending on the number of dashboards on the Looker instance, you may also hit an API rate limit. In that case, uncomment the time.sleep() line. 

Interpreting the output: 
- The script will append data to one of four lists: `next_list`, `legacy_list`, `undefined_list`, or `error_list`. 
- `next_list` = dashboards that are `dashboards-next` and will continue to be next post-21.20
- `legacy_list` = dashboards that are `dashboards-legacy` and will continue to be legacy post-21.20
- `undefined_list` = dashboards that are currently `dashboards-legacy` but do not have a `preferred_viewer` set and will be converted to `dashboards-next` unless action is taken, post-21.20
- `error_list` = dashboards who's metadata could not be parsed due to errors. These dashboards are likely not in use, but the list is included just in case. 

If using colab version, you will need to input your api url, client_id, and client_secret in the `## colab inputs ##` section. 

```
os.environ['LOOKERSDK_BASE_URL'] = 'https://example.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'client id here'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'client secret here'
```

If using python (local) version, these values should be set for the sdk per either an `.ini` file or as an environment variable. See Python SDK documentation for more information. https://pypi.org/project/looker-sdk/

In [ ]:
### colab specific ###
# install the sdk
!pip install looker-sdk
# set environ variables
import os

## colab inputs ##
os.environ['LOOKERSDK_BASE_URL'] = ''
os.environ['LOOKERSDK_API_VERSION'] = ''
os.environ['LOOKERSDK_CLIENT_ID'] = ''
os.environ['LOOKERSDK_CLIENT_SECRET'] = ''

In [ ]:
### import packages ###
# import and init
import time
import looker_sdk
from looker_sdk import models40
sdk = looker_sdk.init40("looker.ini")

all_dashboards = sdk.all_dashboards()
numberofdashboards = len(all_dashboards)

next_list = []
legacy_list = []
undefined_list = []
error_dashboards = []

In [ ]:
for i in range(numberofdashboards):

  try: 
    # uncomment the below line if you hit an api rate limit
    # time.sleep(1)
    current = sdk.dashboard(dashboard_id = all_dashboards[i].id)
    if current.preferred_viewer == "dashboards-next":
      next_list.append(current.id)
    elif current.preferred_viewer == "dashboards":
      legacy_list.append(current.id)
    else:
      undefined_list.append(current.id)
  except:
    error_dashboards.append(all_dashboards[i].id)

In [ ]:
# from here, can either print any of the lists or send to outfile. 
# the list most people will be interested in is `undefined_list`